## 데이터 load 및 확인

In [69]:
import numpy as np
import pandas as pd

color_est = pd.read_csv('../data/color2emotion_rescaling.csv')
emotion_est = pd.read_csv("../result/action_estimation.csv",index_col=0)
action_est = pd.read_csv('../result/emotion_estimation.csv')

In [70]:
color_est[95:105]

,Unnamed: 0,R,G,B,분노,두려움,행복,슬픔,열정,우울
95,95,0,5,215,0.0,0.40,0.00,6.48,0.0,5.50
96,96,0,5,220,0.0,0.41,0.00,6.64,0.0,5.63
97,97,0,5,225,0.0,0.41,0.00,6.61,0.0,5.60
98,98,0,5,230,0.0,0.42,0.00,6.75,0.0,5.73
99,99,0,5,235,0.0,0.41,0.00,6.71,0.0,5.69
100,100,0,5,240,0.0,0.42,0.00,6.85,0.0,5.81
101,101,0,5,245,0.0,0.42,0.00,6.85,0.0,5.81
102,102,0,5,250,0.0,0.43,0.00,6.99,0.0,5.93
103,103,0,5,255,0.0,0.43,0.00,6.98,0.0,5.92
104,104,0,10,0,0.0,0.96,2.84,0.00,0.0,0.00


In [71]:
emotion_est

,에너지,회복,순환,정화
감정,,,,
행복,0.324557,0.344684,0.257535,0.372669
우울,0.256301,0.539817,0.321080,0.273804
슬픔,0.310976,0.308342,0.262105,0.256712
열정,0.269738,0.488793,0.353732,0.360805
분노,0.322202,0.440319,0.408658,0.398532
두려움,0.295698,0.295809,0.393288,0.315617


In [72]:
action_est

,행동,에너지,회복,순환,정화
0,명상,0.359411,0.508459,0.375176,0.364463
1,산책,0.319155,0.540431,0.464981,0.550028
2,등산,0.330991,0.432612,0.464082,0.329396
3,샤워,0.241211,0.316949,0.346316,0.285009
4,수영,0.356064,0.529326,0.358547,0.247034
5,조깅,0.271695,0.426948,0.428360,0.251207
6,자전거 타기,0.322527,0.257293,0.358325,0.177498
7,요가,0.295578,0.374230,0.427756,0.368902
8,스트레칭,0.271916,0.366607,0.424326,0.443960
9,여행,0.305515,0.288080,0.262577,0.317648


In [73]:
import numpy as np
from numpy import dot
from numpy.linalg import norm
from scipy.stats import pearsonr

def pearson_similarity(a, b):
  return np.dot((a - np.mean(a)), (b - np.mean(b))) / ((np.linalg.norm(a - np.mean(a))) * (np.linalg.norm(b - np.mean(b))))

In [140]:
# 색상 -> 감정 -> 효능 <- 행동 시나리오
class color2item():
    def __init__(self , color_df , action_df, emotion_df):
        self.action_df = action_df
        self.emothon_df = emotion_df
        self.color_df = color_df
        self.effs = ['에너지', '회복', '순환', '정화']
             
    def get_color_idx(self,R,G,B):
        R = int(R/5)
        G = int(G/5)
        B = int(B/5)
        return R*52*52 + G*52 + B    
    
    def get_top_ems(self, R,G,B, em_topn):
        em_idx = self.get_color_idx(R,G,B)
        color_dict = {}
        for em in ['분노', '두려움' ,'행복' ,'슬픔' ,'열정','우울']:
            color_dict[em] = self.color_df[em][em_idx]
        color_dict = sorted(color_dict.items() , reverse=True , key = lambda x:x[1])
        
        max_em_dict = {}
        for em_name , em_est in color_dict[:em_topn]:
            max_em_dict[em_name] = em_est
        print(f'색상 {R,G,B}에서 추청치가 높은 {em_topn}개의 감정치{max_em_dict}')
        get_ems_list = list(max_em_dict.keys())
        em_est = self.emothon_df.loc[get_ems_list]
        return em_est
    
    # 각 감정의 추정치에 대해서 각 item의 유사도의 평균
    def each_compare_mean(self, R,G,B, em_topn, eff_topn, ratio=[0.6,0.4]):
        em_est = self.get_top_ems(R,G,B,em_topn)
        
        em_dict = {}
        for em_idx in em_est.index:
            print(f'{em_idx}에 대한 각 item의 유사도 비교(추정치 중 상위 {eff_topn}개를 사용)')
            
            for eff in ['에너지','회복','정화','순환']:
                em_dict[eff] = em_est[eff]
        print(em_dict)
            #em_dict = sorted(em_dict.items() , reverse=True ,key= lambda x:x[1])

            #sim_dict = {} 
            #for dim in em_est.columns:
            #    print(em_idx,dim)
        
        
        return em_est

## 시나리오 테스트 
`색상 -> 감정 -> 효능 -> 상품` 으로 진행  
Setting : 하나의 색상에 대해서 감정 값이 높은 상위 N개를 이용해서 진행 

1. 색상(R,G,B)에서 감정의 추정치가 높은 상위 N개 감정을 선택
2. 선택된 감정에 대해서 추정치가 높은 효능 차원을 선택
3. 상품 데이터의 효능 차원과 감정의 효능 차원을 비교 후 추천

In [141]:
# 내부 과정 시각화
XYZ = color2item(color_est , action_est, emotion_est)
'''
#parameter : R(빨강의 밝기) , G(녹색의 밝기) , B(파랑의 밝기) em_topn(현재 색상에서 추정치가 높은 값을 몇 개로 할지 선택)
XYZ.get_top_ems(0,5,235,2)
'''
XYZ.each_compare_mean(0,5,235,2,1)

# eff_topn(감정에서 추정치가 높은 차원을 몇 개로 할 것인지 선택)

색상 (0, 5, 235)에서 추청치가 높은 2개의 감정치{'슬픔': 6.71, '우울': 5.69}
슬픔에 대한 각 item의 유사도 비교(추정치 중 상위 1개를 사용)
우울에 대한 각 item의 유사도 비교(추정치 중 상위 1개를 사용)
{'에너지': 감정
슬픔    0.310976
우울    0.256301
Name: 에너지, dtype: float64, '회복': 감정
슬픔    0.308342
우울    0.539817
Name: 회복, dtype: float64, '정화': 감정
슬픔    0.256712
우울    0.273804
Name: 정화, dtype: float64, '순환': 감정
슬픔    0.262105
우울    0.321080
Name: 순환, dtype: float64}


,에너지,회복,순환,정화
감정,,,,
슬픔,0.310976,0.308342,0.262105,0.256712
우울,0.256301,0.539817,0.321080,0.273804
